# Analyze cell state dynamics batched experimental conditions
In this tutorial, we will show how to analyze the effect of batched experimental conditions using exdyn. We will apply exdyn to simulated data set where cluster 0 population differentiate into cluster 1 in condition 1 and cluster 2 in condition 2.


## Model set up and optimization
Exdyn accepts AnnData object for dynamics estimation. The model parameters such as the number of neural network layers can be specified through a dictionary of the parameters.

In [ ]:
import scanpy as sc
from exdyn import workflow


orig_adata = sc.read_h5ad('data/sim_adata_mod.h5ad)
model_params = {
            'z_dim': 10,
            'enc_z_h_dim': 50, 'enc_d_h_dim': 50, 'dec_z_h_dim': 50,
            'num_enc_z_layers': 2, 'num_enc_d_layers': 2,
            'num_dec_z_layers': 2, 'use_ambient': False, 'use_vamp': True, 'no_d_kld': False, 'decreasing_temp': False, 'dec_temp_steps': 30000, 'loss_mode': 'nb',
            'dyn_loss_weight': 1
}
checkpoint_dirname = 'checkpoints'
adata, lit_envdyn = workflow.conduct_cvicdyf_inference(orig_adata, model_params, checkpoint_dirname, batch_size=128, two_step=False, dyn_mode=False, epoch=1000, patience=50, module=modules.Cvicdyf, use_highly_variable=False, batch_key='sample', condition_key='condition')

## Visulaize condition wise dynamics
We can visualize cell state dynamics of all the cells in each condition as well as the dynamics in the conditions identical with those where cells were obsserved.

### Cell state dynamics in original conditions

Firstly, we visualize the cell state dynamics in the conditions where each cell originally observed.

In [ ]:
from exdyn import utils
utils.plot_mean_flow(adata, cluster_key='condition', legend_loc='right')

Exdyn can provide an opptunity to derive the conterfactual estimation of cell state dynamics in a condition different from that where the cells were observed. 
Here, we estimate and display the cell state dynamics of all the cells in condition 1 and 2 respectively.

In [ ]:
from exdyn import condiff
cond1 = '1'
cond2 = '2'
conds = [cond1, cond2]
adata = condiff.estimate_two_cond_dynamics(adata, cond1, cond2, lit_envdyn)
for cond in conds:
    utils.plot_mean_flow(adata, cluster_key=None, legend_loc='right', vel_key=f'norm_cond_vel_{cond}', du_key=f'cond_dumap_{cond}')

In general, exdyn can calculate the dynamics across multple conditions more than 2. However, the analysis lacks several two condition specific results.

In [ ]:
multi_adata = condiff.estimate_multi_cond_dynamics(adata, [cond1, cond2], lit_envdyn)
print('Two condition version version:')
print(adata)
print('General version:')
print(multi_adata)

## Identification analysis of bifurction points between conditions 
Exdyn enables us to identify a bifurcation point between the conditions, where the identical cell states have dynamics varying between the econditions. Here, we extract cells with large conditional difference from the cell population shared by both of the conditions.

In [ ]:
adata.obs['Condition difference'] = np.linalg.norm(adata.layers['norm_cond_vel_diff'], axis=1)
common_adata = adata[np.logical_and(adata.obs['Condition 2 ratio'] > 0.3, adata.obs['Condition 2 ratio'] < 0.7)]
top_adata = common_adata[common_adata.obs['Condition difference'] > common_adata.obs['Condition difference'].quantile(0.7)]
sc.pl.umap(adata, color=['Condition difference', 'Top Condition Difference'], palette={'True': 'black', 'False': 'gray'})

adata.obs['Top Condition Difference'] = adata.obs_names.isin(top_adata.obs_names).astype(str)

We can quantify the gene-wise dynamics difference at the bifurcation point, which leads to the identification of the regulator explaining the population difference between the conditions. Here we calculated the conditional difference and visualize them for top difference genes.

In [ ]:
from exdyn import visualization

cdiff_tot_vels = pd.Series(top_adata.layers[f'norm_cond_vel_diff'].mean(axis=0), index=top_adata.var_names)[log2ratio.index]
top_cdiff_vals = pd.concat([cdiff_tot_vels.sort_values(ascending=False)[:5], cdiff_tot_vels.sort_values(ascending=False)[-5:]])
visualization.annotated_bars(ax, top_cdiff_vals.index, top_cdiff_vals.values)
ax.set_ylabel('Conditional dynamics difference')
ax.set_xlabel('Gene')
plt.subplots_adjust(bottom=0.15, left=0.2)

You can conduct clustering on the bifurcation point and analyze subpulation if the identified bifurcation points are heterogeneous.

In [ ]:
sc.tl.leiden(top_adata)
adata.obs['top_leiden'] = None
adata.obs.loc[top_adata.obs_names, 'top_leiden'] = top_adata.obs['leiden']
sc.pl.umap(top_adata, color='top_leiden', legend_loc='on data')
sub_top_adata = adata[adata.obs.top_leiden.isin(['0', '1'])]
